In [1]:
import pandas as pd
import numpy as np

In [28]:
col_names = ["Numero de cuenta", "Oficina", "Divisa", "Fecha operacion", "Fecha valor", 
               "Ingreso (+)", "Gasto (-)", "Saldo (+)", "Saldo (-)", "Concepto comun",
               "Concepto propio", "Referencia 1", "Referencia 2", "Concepto complementario 1",
               "Concepto complementario 2"]

finance = pd.read_csv("finance_2l_utf.csv", header = None, index_col=False, names = col_names_2, sep = ';')

In [29]:
finance.head()

,Numero de cuenta,Oficina,Divisa,Fecha operacion,Fecha valor,Ingreso (+),Gasto (-),Saldo (+),Saldo (-),Concepto comun,Concepto propio,Referencia 1,Referencia 2,Concepto complementario 1,Concepto complementario 2
0,2100 0889 40 0100978503,9736,EUR,17/04/2019,17/04/2019,NaN,"1,73","126,98",NaN,12,40,0,"4,59986E+15",,BON AREA
1,2100 0889 40 0100978503,9792,EUR,17/04/2019,17/04/2019,8,NaN,"128,71",NaN,2,2,0,ORIGEN: 00810900,00810900-ADRIA FERRER ABAD,TRANSF. A SU FAVOR
2,2100 0889 40 0100978503,9792,EUR,17/04/2019,17/04/2019,9,NaN,"120,71",NaN,2,2,0,ORIGEN: 00810053,00810053-PAU SAMPIETRO FABREGAS,TRANSF. A SU FAVOR
3,2100 0889 40 0100978503,9736,EUR,16/04/2019,16/04/2019,NaN,25,"111,71",NaN,12,40,0,"4,59986E+15",,PECUNIA CARDS EDE
4,2100 0889 40 0100978503,569,EUR,16/04/2019,16/04/2019,NaN,"35,08","136,71",NaN,3,1,0,20000005147,A66098435000,AIGUES DE BARC


In [30]:
finance.tail()

,Numero de cuenta,Oficina,Divisa,Fecha operacion,Fecha valor,Ingreso (+),Gasto (-),Saldo (+),Saldo (-),Concepto comun,Concepto propio,Referencia 1,Referencia 2,Concepto complementario 1,Concepto complementario 2
1220,2100 0889 40 0100978503,3002,EUR,01/05/2015,04/05/2015,290,NaN,"324,13",NaN,2,40,0,"4,59986E+15",,INGRESO CAJERO
1221,2100 0889 40 0100978503,9736,EUR,01/05/2015,01/05/2015,NaN,"8,99","34,13",NaN,12,40,0,"4,59986E+15",,PAYPAL *JEANNPOLC
1222,2100 0889 40 0100978503,820,EUR,22/04/2015,22/04/2015,NaN,"10,3","43,12",NaN,17,16,"7,90013E+11",Y1535445W79000,,IMP.AEAT-MOD:790
1223,2100 0889 40 0100978503,820,EUR,22/04/2015,22/04/2015,NaN,"10,6","53,42",NaN,17,16,"7,90013E+11",Y1703074F79000,,IMP.AEAT-MOD:790
1224,2100 0889 40 0100978503,9736,EUR,20/04/2015,20/04/2015,NaN,"7,55","64,02",NaN,12,40,0,"4,59986E+15",,ALIBRI LLIBRERIA


In [31]:
finance.info() # Found some null values in ["ingreso", "gasto"], ["saldo(+)", "saldo(+)"] -> seem to be complementary features

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 15 columns):
Numero de cuenta             1225 non-null object
Oficina                      1225 non-null int64
Divisa                       1225 non-null object
Fecha operacion              1225 non-null object
Fecha valor                  1225 non-null object
Ingreso (+)                  147 non-null object
Gasto (-)                    1078 non-null object
Saldo (+)                    1224 non-null object
Saldo (-)                    1 non-null object
Concepto comun               1225 non-null int64
Concepto propio              1225 non-null int64
Referencia 1                 1225 non-null object
Referencia 2                 1225 non-null object
Concepto complementario 1    1225 non-null object
Concepto complementario 2    1225 non-null object
dtypes: int64(3), object(12)
memory usage: 143.6+ KB


# Drop columns that are not necessary for this analysis

#### Critical data columns that won't be dropped are the following:

#### "Fecha operacion", "Ingreso (+)", "Gasto (-)", "Saldo (+)", "Saldo (-)", "Concepto complementario 1", "Concepto complementario 2"

In [32]:
finance["Numero de cuenta"].value_counts().count() # Account number is always the same because the data is from one single account -> UNNECESSARY

1

#### From now on same account number is assumed -> DROP

In [33]:
finance["Oficina"].value_counts().count()

28

#### There are 28 different offices -> Maybe this feature is useful in the future

In [34]:
finance["Divisa"].value_counts() # Account is european (shared currency EUR) and clearly hasn't had any other currency -> UNNECESSARY

EUR    1225
Name: Divisa, dtype: int64

#### From now on single EUR currency is assumed -> DROP

In [35]:
finance[finance["Fecha operacion"] != finance['Fecha valor']].count()[0] # There are 59

59

#### Some operations were finished pressumably days after the operation request -> Maybe 'Fecha valor' is useful in the future

In [36]:
finance["Concepto comun"].value_counts().head()

12    936
2     145
11     56
4      35
3      35
Name: Concepto comun, dtype: int64

In [37]:
finance["Concepto propio"].value_counts().head()

40    1072
67      49
2       24
1       23
32      17
Name: Concepto propio, dtype: int64

#### Columns "Concepto comun" and "Concepto propio" are not clear what they mean -> UNCLEAR MEANING -> DROP

In [38]:
finance["Referencia 1"].value_counts()

0              1221
7,90013E+11       2
81809083119       1
81802330129       1
Name: Referencia 1, dtype: int64

In [39]:
finance["Referencia 2"].value_counts().head()

4,59986E+15         1069
                      63
20000005147           20
ORIGEN: 01823298      17
Origen: BARCGB22       7
Name: Referencia 2, dtype: int64

#### "Referencia 1" and "Referencia 2" give little to no information -> UNCLEAR MEANING -> DROP

In [40]:
finance["Concepto complementario 1"].value_counts().head()

                                          1136
A66098435000                                20
01823298-MIREIA GOMEZ GORDO                 17
JESSIE LEE DELGADILLO NEWMAN                 8
ROOFOODS SPAIN SL                            7
Name: Concepto complementario 1, dtype: int64

In [41]:
finance["Concepto complementario 2"].value_counts().head()

REINT.CAJERO                              57
LIDL C PUJADES                            54
CAFETERIA DEL MOL                         53
AMAZON.ES COMPRA                          53
MERCADONA RONDA S                         46
Name: Concepto complementario 2, dtype: int64

#### "Concepto complementario 1" is from where a incoming transaction is coming and "Concepto complementario 2" is to where -> Useful

In [42]:
to_drop = ["Numero de cuenta", "Divisa", "Concepto comun", "Concepto propio", "Referencia 1", "Referencia 2"]

finance.drop(to_drop, inplace=True, axis=1)

In [43]:
finance.head() # Result of dropping

,Oficina,Fecha operacion,Fecha valor,Ingreso (+),Gasto (-),Saldo (+),Saldo (-),Concepto complementario 1,Concepto complementario 2
0,9736,17/04/2019,17/04/2019,NaN,"1,73","126,98",NaN,,BON AREA
1,9792,17/04/2019,17/04/2019,8,NaN,"128,71",NaN,00810900-ADRIA FERRER ABAD,TRANSF. A SU FAVOR
2,9792,17/04/2019,17/04/2019,9,NaN,"120,71",NaN,00810053-PAU SAMPIETRO FABREGAS,TRANSF. A SU FAVOR
3,9736,16/04/2019,16/04/2019,NaN,25,"111,71",NaN,,PECUNIA CARDS EDE
4,569,16/04/2019,16/04/2019,NaN,"35,08","136,71",NaN,A66098435000,AIGUES DE BARC


# Convert types

#### Convert "Fecha operacion" and "Fecha valor" to datetime format

In [44]:
finance["Fecha operacion"] = pd.to_datetime(finance["Fecha operacion"], format='%d/%m/%Y')

In [45]:
finance["Fecha valor"] = pd.to_datetime(finance["Fecha valor"], format='%d/%m/%Y')

#### Convert "Ingreso (+)", "Gasto (-)", "Saldo (+)" and "Saldo (-)" to numeric value

In [46]:
def banking_to_numeric(df, column):
    # Replace dots with blankspace (1.000 -> 1000)
    df[column] = df[column].str.replace('.', '', regex=True)
    
    # Replace commas for dots (1,5 -> 1.5)
    df[column] = df[column].str.replace(',', '.', regex=True)
    
    # Convert to numeric
    df[column] = pd.to_numeric(df[column], errors='raise')

In [47]:
to_num_col = ["Ingreso (+)", "Gasto (-)", "Saldo (+)", "Saldo (-)"]

for col in to_num_col:
    banking_to_numeric(finance, col)

In [49]:
finance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 9 columns):
Oficina                      1225 non-null int64
Fecha operacion              1225 non-null datetime64[ns]
Fecha valor                  1225 non-null datetime64[ns]
Ingreso (+)                  147 non-null float64
Gasto (-)                    1078 non-null float64
Saldo (+)                    1224 non-null float64
Saldo (-)                    1 non-null float64
Concepto complementario 1    1225 non-null object
Concepto complementario 2    1225 non-null object
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 86.2+ KB


# Rename columns to more useful ones

In [51]:
new_names =  {'Concepto complementario 1': 'Concepto origen',
              'Concepto complementario 2': 'Concepto destino'}

finance.rename(columns=new_names, inplace=True)

In [53]:
finance.columns

Index(['Oficina', 'Fecha operacion', 'Fecha valor', 'Ingreso (+)', 'Gasto (-)',
       'Saldo (+)', 'Saldo (-)', 'Concepto origen', 'Concepto destino'],
      dtype='object')

In [54]:
finance.to_csv('finance_clean.csv')